In [1]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from data import *
from objectives import *
from plots import *
import sys
import time
import datetime
sys.path.append('/home/hao/Research/probtorch/')
from torch.distributions.uniform import Uniform
from torch.distributions.normal import Normal
import probtorch
from probtorch.util import expand_inputs

print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [ ]:
T = 50
D = 4
NUM_OBS = 2 * D
NUM_SAMPLES = 10
## Model Parameters
NUM_HIDDEN = 64
NUM_LATENT = D 
BATCH_SIZE = 20
NUM_EPOCHS = 100
LEARNING_RATE = 1e-3
CUDA = torch.cuda.is_available()

In [ ]:
Ys = np.load('signals/sequences.npy')

In [ ]:
class Encoder(nn.Module):
    def __init__(self, num_obs=NUM_OBS,
                       num_hidden=NUM_HIDDEN,
                       num_latents=NUM_LATENT):
        super(self.__class__, self).__init__()
        self.enc_hidden = nn.Sequential(
            nn.Linear(num_obs, num_hidden),
            nn.Tanh())
        self.latent_mean = nn.Sequential(
            nn.Linear(num_hidden, num_latents))
        self.latent_log_std = nn.Sequential(
            nn.Linear(num_hidden, num_latents))
        
    def forward(self, obs, batch_size):
        hidden = self.enc_hidden(obs)
        mean = self.latent_mean(hidden).view(batch_size, T-1, D).sum(1).view(batch_size, D)
        std = torch.exp(self.latent_log_std(hidden).view(batch_size, T-1, D).sum(1).view(batch_size, D))
        return mean, std

In [ ]:
def initialize():
    enc = Encoder()
    if CUDA:
        enc.cuda()
    optimizer =  torch.optim.Adam(list(enc.parameters()),lr=LEARNING_RATE)    
    return enc, optimizer
enc, optimizer = initialize()

In [ ]:
def log_ps_lds(As, data, num_samples, T, D, batch_size):
    ## As_expanded is SB-D-D
    cov_noise = (torch.ones(D) * (0.02 ** 2)).cuda()
    ## As S-B-D
    log_p = torch.zeros((num_samples, batch_size)).cuda()
    x0 = data[:, 0, :].unsqueeze(0).repeat(num_samples, 1, 1)
    log_p = log_p + Uniform((torch.ones(D)*(-1)).cuda(), torch.ones(D).cuda()).log_prob(x0).sum(-1)
    for t in range(T-1):
        ## S-B-D
        xt = data[:, t, :].unsqueeze(0).repeat(num_samples, 1, 1)
        xtp1 = data[:, t+1, :].unsqueeze(0).repeat(num_samples, 1, 1)
        mus = torch.mul(As, xt)
        log_p = log_p + Normal(mus, cov_noise).log_prob(xtp1).sum(-1)
    return log_p

In [ ]:
Grad_Steps = int((Ys.shape[0] / BATCH_SIZE))

for epoch in range(NUM_EPOCHS):
    np.random.shuffle(Ys)
    for steps in range(Grad_Steps):
        time_start = time.time()
        batch = torch.from_numpy(Ys[:BATCH_SIZE, :, :]).float()
        batch_flat = torch.cat((batch[:, :T-1, :].unsqueeze(2), batch[:, 1:, :].unsqueeze(2)), 2).view(BATCH_SIZE * (T-1), 2*D)
        batch_flat = batch_flat.cuda()
        batch = batch.cuda()
        optimizer.zero_grad()
        mean, std = enc(batch_flat, BATCH_SIZE)
        epsilons = Normal(torch.zeros(D), torch.ones(D)).sample((NUM_SAMPLES, BATCH_SIZE)).cuda()
        As = (torch.mul(epsilons, std.unsqueeze(0).repeat(NUM_SAMPLES, 1, 1)) + mean.unsqueeze(0).repeat(NUM_SAMPLES, 1, 1)).detach()
        log_q_As = Normal(mean, std).log_prob(As).sum(-1)
        prior_mean = torch.zeros(D).cuda()
        prior_std = torch.ones(D).cuda()
        log_p_As = Normal(prior_mean, prior_std).log_prob(As).sum(-1)
        log_p_xs = log_ps_lds(As, batch, NUM_SAMPLES, T, D, BATCH_SIZE)
        elbo = (log_p_xs.detach() + log_p_As.detach() - log_q_As).mean(0).mean(0)
        loss = -elbo
        loss.backward()
        optimizer.step()
        time_end = time.time()
        print('epoch : %d, ELBO : %f (%ds)' % (epoch, elbo, time_end - time_start))

In [ ]:
elbo

In [ ]:
plot_results(EUBOs, ELBOs, ESSs, KLs, 'results_sis.png')

In [2]:
np.random.uniform(1, 2, 3)

array([1.81194319, 1.62139272, 1.59213494])

In [ ]:
x0